# EMA Crossover Strategy Data Preprocessing

In [8]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import random
import requests
from bs4 import BeautifulSoup
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import mplfinance as mpf
from lightgbm import LGBMClassifier

In [2]:
def get_sp500_tickers():
    url = 'https://www.slickcharts.com/sp500'
    html = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}).text
    table = pd.read_html(str(BeautifulSoup(html, 'lxml').find('table')))[0]
    return table['Symbol'].str.replace('.', '-', regex=False).tolist()

tickers = get_sp500_tickers()
df = yf.download(tickers, start='2010-01-01', end='2025-04-22', group_by='ticker', auto_adjust=True)

C:\Users\rusla\AppData\Local\Temp\ipykernel_22476\631629297.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(BeautifulSoup(html, 'lxml').find('table')))[0]
[*********************100%***********************]  503 of 503 completed


In [3]:
# Multi to singlular column transformation
df.columns = df.columns.swaplevel(0, 1)
df_flat = df.stack(level=1, future_stack=True).reset_index()
df_flat.columns.name = None
df_flat.columns = [col if not isinstance(col, tuple) else col[-1] for col in df_flat.columns]
df_flat.head()

,Date,Ticker,Open,High,Low,Close,Volume
0,2010-01-04,CEG,NaN,NaN,NaN,NaN,NaN
1,2010-01-04,OTIS,NaN,NaN,NaN,NaN,NaN
2,2010-01-04,FTV,NaN,NaN,NaN,NaN,NaN
3,2010-01-04,IRM,8.628732,8.692816,8.549569,8.628732,995548.0
4,2010-01-04,IR,NaN,NaN,NaN,NaN,NaN


In [4]:
df_flat.dropna(inplace=True)
df_flat.isnull().sum()

Date      0
Ticker    0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

## Technical Indicators

- __RSI (Relative Strength Index):__ Measures the speed and magnitude of recent price changes to identify overbought (>70) or oversold (<30) conditions.

    - If RSI > 50 then uptrend is gaining strength
    - If RSI < 50 then momentum is weakening

<br>

- __MACD (Moving Average Convergence Divergence):__ Captures the difference between short and long term momentum via EMAs.

    - MACD = EMA12 - EMA26
    - Signal line = EMA9 of MACD
    - macd > signal = bullish momentum

<br>

- __ATR (Average True Range):__  Is a volatility indicator that reflects the average price variation of an asset.

    - Calculates the maximum of high-low, high-previous close, and low-previous close
    - Averages this over a rolling window, usually set to 14 days
    - Higher ATR can indicate breakout potential or stop-loss placement zones

<br>

- __CCI (Commodity Channel Index):__ Detects price deviations from a statistical mean.

    - Computes the typical price = (High + Low + Close) / 3
    - Measures how far the typical price deviates from its moving average
    - CCI > +100 suggests strong bullish deviation
    - CCI < -100 implies a potential trend reversal

<br>

- __HMA (Hull Moving Average):__ Is a smoother, faster reacting moving average used to reduce lag.

    - Combines weighted averages of two rolling means
    - Applies a square root weighted smoothing

In [5]:
def compute_rsi(series, window=14):
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.ewm(span=window, adjust=False).mean()
    avg_loss = loss.ewm(span=window, adjust=False).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_macd(df):
    ema12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema26 = df['Close'].ewm(span=26, adjust=False).mean()
    macd = ema12 - ema26
    signal = macd.ewm(span=9, adjust=False).mean()
    return macd, signal

def calculate_atr(df, period=14):
    high_low = df['High'] - df['Low']
    high_close = (df['High'] - df['Close'].shift()).abs()
    low_close = (df['Low'] - df['Close'].shift()).abs()
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    atr = tr.rolling(window=period).mean()
    return atr

def calculate_cci(df, period=20):
    tp = (df['High'] + df['Low'] + df['Close']) / 3
    sma = tp.rolling(window=period).mean()
    mad = tp.rolling(window=period).apply(lambda x: np.mean(np.abs(x - np.mean(x))))
    cci = (tp - sma) / (0.015 * mad)
    return cci

def calculate_hma(close, period=20):
    half = close.rolling(window=period//2).mean()
    full = close.rolling(window=period).mean()
    raw_hma = 2 * half - full
    hma = raw_hma.rolling(window=int(np.sqrt(period))).mean()
    return hma


## Feature Engineering

- __EMA10, EMA20, EMA50, EMA200:__ Exponential moving averages over different spans, these provide smoothed price trends, with the 50/200 pair used to detect "golden cross" signals.

- __momentum_5:__ Percentage change over the last 5 days, a simple measure of short term momentum or trend acceleration.

- __ema_distance:__ Difference between EMA50 and EMA200, helps quantify how far the two EMAs have diverged.

- __ema50_slope:__ 5 day slope of the EMA50 line, provides insight into the trajectory and speed of trend development.

- __price_ema200_dist:__ Distance between the current close and EMA200.

- __EMA50_prev/EMA200_prev:__ Shifted versions of EMA50 and EMA200, used to identify crossovers.

- __RSI_14:__ 14 day Relative Strength Index, gauges overbought or oversold conditions.

- __MACD/MACD_signal:__ MACD and its signal line.

- __macd_diff:__ Difference between MACD and its signal line, indicates strength of the MACD trend.

- __ATR_14:__ Average True Range over 14 days, measuring volatility.

- __CCI_20:__ Commodity Channel Index over 20 days, another momentum indicator.

- __BB_MID/BB_UPPER/BB_LOWER:__ Bollinger Bands based on 20 day mean and standard deviation.

- __HMA_20:__ Hull Moving Average with 20 period window.

<br>

__Labeling Strategy:__ Used a hybrid logic for labeling crossover signals as profitable (1) or not profitable (0)
- __forward_return:__ The 10 day forward return is calculated as the percentage return from the current date to 10 trading days ahead.
- __Label = 1:__
    - If the forward return exceeds +3% OR
    - If the crossover is supported by bullish MACD and RSI momentum (MACD > MACD Signal & RSI > 50)
- __Label = 0:__ All other cases

- *Why did I end up using this strategy over a standard label that took `forward_return` > 0.3?*
    - It mislabeled "slow" crossovers that took longer to become profitable as "0"
    - Mislabeled "bad" crossovers as good due to short term volatility
    - Saw many signals labeled as "bad" were actually good in hindsight if held longer, also saw "good" labeled signals showing brief momentum and then failing


In [6]:
# FEATURE ENGINEERING
df_flat['EMA10'] = df_flat.groupby('Ticker')['Close'].transform(lambda x: x.ewm(span=10, adjust=False).mean())
df_flat['EMA20'] = df_flat.groupby('Ticker')['Close'].transform(lambda x: x.ewm(span=20, adjust=False).mean())
df_flat['EMA50'] = df_flat.groupby('Ticker')['Close'].transform(lambda x: x.ewm(span=50).mean())
df_flat['EMA200'] = df_flat.groupby('Ticker')['Close'].transform(lambda x: x.ewm(span=200).mean())

df_flat['momentum_5'] = df_flat.groupby('Ticker')['Close'].pct_change(5, fill_method=None)

df_flat['ema_distance'] = df_flat['EMA50'] - df_flat['EMA200']

df_flat['ema50_slope'] = df_flat.groupby('Ticker')['EMA50'].diff(5)

df_flat['price_ema200_dist'] = df_flat['Close'] - df_flat['EMA200']

df_flat['EMA50_prev'] = df_flat.groupby('Ticker')['EMA50'].shift(1)
df_flat['EMA200_prev'] = df_flat.groupby('Ticker')['EMA200'].shift(1)

df_flat['RSI_14'] = df_flat.groupby('Ticker')['Close'].transform(lambda x: compute_rsi(x))

df_flat[['MACD', 'MACD_signal']] = df_flat.groupby('Ticker').apply(
    lambda x: pd.DataFrame({
        'MACD': calculate_macd(x)[0],
        'MACD_signal': calculate_macd(x)[1]
    }, index=x.index)
).reset_index(drop=True)

df_flat['macd_diff'] = df_flat['MACD'] - df_flat['MACD_signal']

df_flat['ATR_14'] = df_flat.groupby('Ticker').apply(calculate_atr).reset_index(level=0, drop=True)

df_flat['CCI_20'] = df_flat.groupby('Ticker').apply(calculate_cci).reset_index(level=0, drop=True)

df_flat['BB_MID'] = df_flat.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=20).mean())
df_flat['BB_STD'] = df_flat.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=20).std())
df_flat['BB_UPPER'] = df_flat['BB_MID'] + 2 * df_flat['BB_STD']
df_flat['BB_LOWER'] = df_flat['BB_MID'] - 2 * df_flat['BB_STD']

df_flat['HMA_20'] = df_flat.groupby('Ticker')['Close'].transform(calculate_hma)

df_flat['crossover'] = (
    (df_flat['EMA50_prev'] < df_flat['EMA200_prev']) &
    (df_flat['EMA50'] > df_flat['EMA200'])
)

df_flat['forward_return'] = df_flat.groupby('Ticker')['Close'].shift(-10) / df_flat['Close'] - 1


df_flat['label'] = np.where(
    (df_flat['forward_return'] > 0.03),
    1,
    np.where(
        (df_flat['MACD'] > df_flat['MACD_signal']) & (df_flat['RSI_14'] > 50),
        1,
        0
    )
)   

C:\Users\rusla\AppData\Local\Temp\ipykernel_22476\1616348723.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_flat[['MACD', 'MACD_signal']] = df_flat.groupby('Ticker').apply(
C:\Users\rusla\AppData\Local\Temp\ipykernel_22476\1616348723.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_flat['ATR_14'] = df_flat.groupby('Ticker').apply(calculate_atr).reset_index(level=0, drop=True)
C:\Users\rusla\

## Feature Preparation
Don't need to apply dummy encoding for the `'Ticker'` column because this model will be trained on one ticker at a time. 

Likewise, it's unnecessaryto do any standard feature scaling because:
- All input features are relative, technical indicators which are already on similar scales.
- It simplifies preprocessing without compromising model accuracy.

In [9]:
df_model = df_flat[df_flat.crossover].copy()

features = [
    'EMA10', 'EMA20', 'EMA50', 'EMA200',
    'momentum_5', 'ema_distance', 'ema50_slope',
    'price_ema200_dist', 'RSI_14', 'MACD', 'MACD_signal', 'macd_diff',
    'ATR_14', 'CCI_20', 'BB_MID', 'BB_STD', 'BB_UPPER', 'BB_LOWER',
    'HMA_20'
]

X = df_model[features]
y = df_model['label']

## Train/Test Split

Split the dataset into training and test sets using `train_test_split()` with the following parameters:
- 70% training, 30% testing
- Stratified by the target variable `label` to ensure balanced class distribution in both the training and test sets
- A fixed `random_state` for reproducibility

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)